In [1]:
import numpy as np
import edf

In [2]:
ops = edf.ops
params = edf.params
values = edf.values

# Average pooling
class avg_pool:
    def __init__(self,x,sz):
        ops.append(self)
        self.x = x
        self.sz = sz

    def forward(self):
        B,H,W,C = self.x.top.shape
        top = np.zeros([B,H-self.sz+1,W-self.sz+1,C])
        for i in range(self.sz):
            for j in range(self.sz):
                xcrop = self.x.top[:, i:(H-self.sz+1+i), j:(W-self.sz+1+j), :].copy()
                top = top + xcrop
        
        self.top = top / np.float32(self.sz*self.sz)
        print(self.x.top[0,:,:,0])
        print(self.top[0,:,:,0])

    def backward(self):
        if self.x in ops or self.x in params:
            B,H,W,C = self.x.top.shape
            xgrad = np.zeros([B,H,W,C])
            for i in range(self.sz):
                for j in range(self.sz):
                    xgrad[:, i:(H-self.sz+1+i), j:(W-self.sz+1+j), :] += self.grad / (self.sz*self.sz)

            self.x.grad = self.x.grad + xgrad
            print(self.grad[0,:,:,0])
            print(self.x.grad[0,:,:,0])
            
            
edf.avg_pool = avg_pool

In [3]:
#######################################

# Inputs and parameters
inp = edf.Param()
lab = edf.Value()

# Model
y = edf.avg_pool(inp,2)

# Loss
loss = edf.add(y,lab)

In [4]:
# Forward test
data = np.arange(50).reshape([1,5,5,2])
inp.set(data)
l = np.ones([1,4,4,2])*(-1.0)
lab.set(l)

edf.Forward()

[[ 0.  2.  4.  6.  8.]
 [10. 12. 14. 16. 18.]
 [20. 22. 24. 26. 28.]
 [30. 32. 34. 36. 38.]
 [40. 42. 44. 46. 48.]]
[[ 6.  8. 10. 12.]
 [16. 18. 20. 22.]
 [26. 28. 30. 32.]
 [36. 38. 40. 42.]]


In [5]:
# Backward test
edf.Backward(loss)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[0.25 0.5  0.5  0.5  0.25]
 [0.5  1.   1.   1.   0.5 ]
 [0.5  1.   1.   1.   0.5 ]
 [0.5  1.   1.   1.   0.5 ]
 [0.25 0.5  0.5  0.5  0.25]]
